In [ ]:
!pip install m3gnet

In [ ]:
!pip install tensorflow==2.13.1


In [ ]:
!pip install numpy==1.26.4

In [ ]:
# Mount Google Drive in Colab
#!pip install matgl
from google.colab import drive
drive.mount('/content/drive')

import os
import csv
from ase.constraints import FixAtoms
from ase.io import read, write
from ase.mep import NEB
from ase.optimize import BFGS, QuasiNewton
from m3gnet.models import M3GNet, Potential, M3GNetCalculator, Relaxer
# Set device to GPU (A100)
device = 'gpu'  # or 'cuda:0' if preferred
base_input_dir = '/content/drive/MyDrive/MLIP_NEB_inputs'

# Walk through the directory tree under MLIP_NEB_inputs
for root, dirs, files in os.walk(base_input_dir):
    # Check if both "initial" and "final" folders are present in the current directory
    if 'initial' in dirs and 'final' in dirs:
        initial_path = os.path.join(root, "initial", "POSCAR")
        final_path = os.path.join(root, "final", "POSCAR")

        # Only process if both POSCAR files exist
        if not (os.path.exists(initial_path) and os.path.exists(final_path)):
            print(f"Skipping {root}: initial or final POSCAR not found.")
            continue

        system_folder = root  # This is the system's folder
        system_name = os.path.basename(system_folder)
        print(f"Processing system: {system_name}")

        # Create a subfolder in the system folder for the NEB outputs
        save_dir = os.path.join(system_folder, "M3GNet_RUN")
        os.makedirs(save_dir, exist_ok=True)

        # CSV file to record energies
        csv_path = os.path.join(save_dir, "energies.csv")

        # Read initial and final structures
        initial = read(initial_path)
        final = read(final_path)

        # Apply constraints: fix atoms with tag > 1
        mask_initial = [atom.tag > 1 for atom in initial]
        initial.set_constraint(FixAtoms(mask=mask_initial))
        mask_final = [atom.tag > 1 for atom in final]
        final.set_constraint(FixAtoms(mask=mask_final))
        # We'll use the final mask for the NEB images

        # Set up M3GNet calculator and assign it to both endpoints
        potential = Potential(M3GNet.load())
        calc = M3GNetCalculator(potential=potential)
        initial.calc = calc
        final.calc = calc

        # Relax the initial structure
        init_traj = os.path.join(save_dir, "initial_relaxed.traj")
        qn = QuasiNewton(initial, trajectory=init_traj)
        qn.run(fmax=0.05, steps=1000)
        write(os.path.join(save_dir, "relaxed_initial.vasp"), initial, format="vasp")

        # Relax the final structure
        final_traj = os.path.join(save_dir, "final_relaxed.traj")
        qn = QuasiNewton(final, trajectory=final_traj)
        qn.run(fmax=0.05, steps=1000)
        write(os.path.join(save_dir, "relaxed_final.vasp"), final, format="vasp")

        # Reload relaxed structures
        initial = read(init_traj)
        final = read(final_traj)

        # Create NEB images (7 intermediate images)
        constraint = FixAtoms(mask=mask_final)
        images = [initial]
        for i in range(7):
            image = initial.copy()
            # Assign a fresh CHGNet calculator to each image
            potential = Potential(M3GNet.load())
            calc = M3GNetCalculator(potential=potential)
            image.set_constraint(constraint)
            images.append(image)
        images.append(final)

        # Set up and run the NEB calculation
        neb = NEB(images, parallel=True, k=5, method='eb')
        neb.interpolate('idpp')
        neb_traj = os.path.join(save_dir, "neb.traj")
        qn = BFGS(neb, trajectory=neb_traj)
        qn.run(fmax=0.05, steps=1000)

        # Compute and save energies for each image
        energies = []
        with open(csv_path, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["Image Index", "Energy (eV)"])
            for i, image in enumerate(images):
                try:
                    energy = image.get_potential_energy()
                except Exception as e:
                    print(f"Error computing energy for image {i} of {system_name}: {e}")
                    energy = None
                energies.append(energy)
                output_file = os.path.join(save_dir, f"neb_optimized_{i}.vasp")
                write(output_file, image, format="vasp")
                writer.writerow([i, energy])

        # Check for any energy errors before calculating migration barrier
        if None in energies:
            print(f"Skipping migration barrier calculation for {system_name} due to energy errors")
            continue

        migration_barrier = max(energies) - min(energies)
        print(f"{system_name} - Migration Barrier: {migration_barrier:.6f} eV")
        with open(csv_path, mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([])
            writer.writerow(["Migration Barrier", migration_barrier])

print("All NEB Calculations complete!")


